In [7]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.5 MB/s eta 0:00:00


In [9]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 15.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatibl

In [21]:
import networkx as nx
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.transforms import RandomLinkSplit
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec
import itertools
from collections import Counter
import io

In [2]:
countries = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia",
    "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium",
    "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria",
    "Burkina Faso", "Burundi", "Cambodia", "Cameroon", "Canada", "Cape Verde", "Central African Republic", "Chad",
    "Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", "Côte d'Ivoire", "Croatia", "Cuba", "Cyprus",
    "Czech Republic", "North Korea", "Democratic Republic of the Congo", "Denmark", "Djibouti", "Dominica",
    "Dominican Republic", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini",
    "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada",
    "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "Hungary", "Iceland", "India",
    "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya",
    "Kiribati", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein",
    "Lithuania", "Luxembourg", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands",
    "Mauritania", "Mauritius", "Mexico", "Micronesia", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique",
    "Myanmar", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria",
    "North Macedonia", "Norway", "Oman", "Pakistan", "Palau", "Panama", "Papua New Guinea", "Paraguay", "Peru",
    "Philippines", "Poland", "Portugal", "Qatar", "South Korea", "Moldova", "Romania", "Russia", "Rwanda",
    "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", "Samoa", "San Marino",
    "São Tomé and Príncipe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore",
    "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain", "Sri Lanka",
    "Sudan", "Suriname", "Sweden", "Switzerland", "Syria", "Tajikistan", "Tanzania", "Thailand", "Timor-Leste",
    "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine",
    "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Venezuela",
    "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

atlas_graph = nx.DiGraph()
atlas_graph.add_nodes_from(countries)
edges = [
    (c1, c2) for c1, c2 in itertools.product(countries, repeat=2)
    if c1 != c2 and c1.lower()[-1] == c2.lower()[0]
]
atlas_graph.add_edges_from(edges)

In [3]:
class CountryPopularityScorer:
    def __init__(self, country_list):
        # Sources: This data is representative of figures from sources like The World Bank (Population, GDP) and cultural influence rankings (e.g., U.S. News Best Countries Report).
        self.population_data = {
            'China': 1440, 'India': 1380, 'United States': 331, 'Indonesia': 273, 'Pakistan': 225, 'Brazil': 213,
            'Nigeria': 211, 'Bangladesh': 165, 'Russia': 146, 'Mexico': 129, 'Japan': 125, 'Germany': 83,
            'United Kingdom': 68, 'France': 68, 'Italy': 60, 'South Africa': 59, 'South Korea': 52,
            'Colombia': 51, 'Spain': 47, 'Argentina': 45, 'Canada': 38, 'Saudi Arabia': 35, 'Australia': 25
        }
        self.economic_prominence = {
            'United States': 10, 'China': 9, 'Japan': 8, 'Germany': 8, 'India': 7, 'United Kingdom': 7,
            'France': 7, 'Italy': 6, 'Brazil': 6, 'Canada': 6, 'Russia': 6, 'South Korea': 6, 'Australia': 5,
            'Spain': 5, 'Mexico': 5, 'Indonesia': 5, 'Netherlands': 5, 'Saudi Arabia': 5, 'Turkey': 4,
            'Switzerland': 4, 'Argentina': 4, 'Nigeria': 4
        }
        self.cultural_prominence = {
            'United States': 10, 'United Kingdom': 8, 'France': 7, 'Italy': 7, 'Germany': 6, 'Japan': 8,
            'Australia': 6, 'Canada': 6, 'Spain': 6, 'China': 7, 'India': 8, 'Brazil': 7, 'Russia': 6,
            'Mexico': 6, 'South Korea': 8, 'Egypt': 5, 'Greece': 5, 'Turkey': 5
        }
        self.all_countries = country_list
        self.popularity_scores = self._calculate_popularity_scores()

    def _calculate_popularity_scores(self):
        scores = {}
        max_pop = max(self.population_data.values())

        for country in self.all_countries:
            pop_score = np.log1p(self.population_data.get(country, 0)) / np.log1p(max_pop)

            econ_score = self.economic_prominence.get(country, 1) / 10.0
            cult_score = self.cultural_prominence.get(country, 1) / 10.0

            combined_score = (pop_score * 0.2) + (econ_score * 0.4) + (cult_score * 0.4)
            scores[country] = combined_score

        max_score = max(scores.values())
        min_score = min(scores.values())
        final_scores = {k: (v - min_score) / (max_score - min_score) for k, v in scores.items()}
        return final_scores

    def get_score(self, country):
        return self.popularity_scores.get(country, 0)

popularity_scorer = CountryPopularityScorer(countries)

In [18]:
class AtlasFeatureEngineer:
    def __init__(self, graph, country_list, scorer):
        self.graph = graph
        self.countries = country_list
        self.scorer = scorer

    def create_combined_features(self, normalize=True):
        letter_features = self._create_letter_features()
        structural_features = self._create_structural_features()
        linguistic_features = self._create_linguistic_features()
        popularity_features = self._create_popularity_features()
        combined = np.concatenate([
            letter_features, structural_features, linguistic_features, popularity_features
        ], axis=1)
        if normalize:
            scaler = StandardScaler()
            combined = scaler.fit_transform(combined)
        return combined

    def _create_letter_features(self):
        all_letters = sorted(list(set(c[0].lower() for c in self.countries) | set(c[-1].lower() for c in self.countries)))
        letter_map = {letter: i for i, letter in enumerate(all_letters)}
        num_letters = len(all_letters)
        features = []
        for country in self.countries:
            first, last = [0] * num_letters, [0] * num_letters
            first[letter_map[country[0].lower()]] = 1
            last[letter_map[country[-1].lower()]] = 1
            features.append(first + last)
        return np.array(features)

    def _create_structural_features(self):
        pagerank = nx.pagerank(self.graph)
        betweenness = nx.betweenness_centrality(self.graph)
        features = []
        for country in self.countries:
            features.append([
                self.graph.in_degree(country), self.graph.out_degree(country),
                pagerank.get(country, 0), betweenness.get(country, 0)
            ])
        return np.array(features)

    def _create_linguistic_features(self):
        vowels = set('aeiou')
        features = []
        for country in self.countries:
            name = country.lower()
            features.append([len(name), len(name.replace(' ', '')), sum(1 for char in name if char in vowels)])
        return np.array(features)

    def _create_popularity_features(self):
        return np.array([[self.scorer.get_score(country)] for country in self.countries])

feature_engineer = AtlasFeatureEngineer(atlas_graph, countries, popularity_scorer)
node_features = feature_engineer.create_combined_features()

In [19]:
class Node2VecLinkPredictor:
    def __init__(self, graph, dimensions=64, walk_length=20, num_walks=100):
        self.graph = graph
        self.dimensions = dimensions
        self.walk_length = walk_length
        self.num_walks = num_walks
        self.model = None

    def train(self):
        walks = self._generate_walks()
        self.model = Word2Vec(walks, vector_size=self.dimensions, window=5, min_count=1, sg=1, workers=4)

    def get_embeddings(self):
        return np.array([self.model.wv[node] for node in self.graph.nodes()])

    def _generate_walks(self):
        walks = []
        for _ in range(self.num_walks):
            for node in self.graph.nodes():
                walk = [node]
                while len(walk) < self.walk_length:
                    current = walk[-1]
                    neighbors = list(self.graph.successors(current))
                    if len(neighbors) > 0:
                        walk.append(np.random.choice(neighbors))
                    else:
                        break
                walks.append(walk)
        return walks

class GNNLinkPredictor(nn.Module):
    def __init__(self, num_features, hidden_dim=64, num_layers=2, heads=4, dropout=0.2):
        super().__init__()
        self.convs = nn.ModuleList()
        self.convs.append(GATConv(num_features, hidden_dim, heads=heads, dropout=dropout))
        if num_layers > 1:
            self.convs.append(GATConv(hidden_dim * heads, hidden_dim, heads=1, concat=False, dropout=dropout))

        self.link_predictor = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)
        )

    def encode(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < len(self.convs) - 1:
                x = F.elu(x)
                x = F.dropout(x, p=0.6, training=self.training)
        return x

    def decode(self, z, edge_label_index):
        edge_embeddings = torch.cat([z[edge_label_index[0]], z[edge_label_index[1]]], dim=-1)
        return self.link_predictor(edge_embeddings).squeeze()

In [24]:
features_tensor = torch.tensor(node_features, dtype=torch.float)
edge_index = torch.tensor([[list(atlas_graph.nodes()).index(u), list(atlas_graph.nodes()).index(v)] for u, v in atlas_graph.edges()], dtype=torch.long).t().contiguous()
data = Data(x=features_tensor, edge_index=edge_index)
transform = RandomLinkSplit(num_val=0.1, num_test=0.1, is_undirected=False, add_negative_train_samples=True)
train_data, val_data, test_data = transform(data)

# Evaluate Node2Vec
print("Training and Evaluating Model 1: Node2Vec")
node2vec = Node2VecLinkPredictor(atlas_graph)
print("Training Node2Vec model")
node2vec.train()
print("Node2Vec training complete.")
embeddings = node2vec.get_embeddings()

train_pos_edges = train_data.edge_label_index[:, train_data.edge_label == 1]
train_neg_edges = train_data.edge_label_index[:, train_data.edge_label == 0]
pos_train_feat = embeddings[train_pos_edges[0]] * embeddings[train_pos_edges[1]]
neg_train_feat = embeddings[train_neg_edges[0]] * embeddings[train_neg_edges[1]]
X_train = np.vstack([pos_train_feat, neg_train_feat])
y_train = np.hstack([np.ones(pos_train_feat.shape[0]), np.zeros(neg_train_feat.shape[0])])

lr = LogisticRegression(solver='liblinear', max_iter=1000)
print("Training a classifier on Node2Vec embeddings")
lr.fit(X_train, y_train)
print("Classifier training complete.")

test_pos_edges = test_data.edge_label_index[:, test_data.edge_label == 1]
test_neg_edges = test_data.edge_label_index[:, test_data.edge_label == 0]
pos_test_feat = embeddings[test_pos_edges[0]] * embeddings[test_pos_edges[1]]
neg_test_feat = embeddings[test_neg_edges[0]] * embeddings[test_neg_edges[1]]
X_test = np.vstack([pos_test_feat, neg_test_feat])
y_test = np.hstack([np.ones(pos_test_feat.shape[0]), np.zeros(neg_test_feat.shape[0])])

test_probs = lr.predict_proba(X_test)[:, 1]
n2v_auc = roc_auc_score(y_test, test_probs)
n2v_ap = average_precision_score(y_test, test_probs)

# Evaluate GNN
print("Training and Evaluating Model 2: GNN (Graph Attention Network)")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNLinkPredictor(data.num_features).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x.to(device), train_data.edge_index.to(device))
    out = model.decode(z, train_data.edge_label_index.to(device))
    loss = criterion(out, train_data.edge_label.to(device).float())
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x.to(device), data.edge_index.to(device))
    out = model.decode(z, data.edge_label_index.to(device))
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

for epoch in range(1, 201):
    loss = train()
    if epoch % 20 == 0:
        val_auc = test(val_data)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AUC: {val_auc:.4f}')

gnn_auc = test(test_data)
z_gnn = model.encode(test_data.x.to(device), test_data.edge_index.to(device))
out_gnn = model.decode(z_gnn, test_data.edge_label_index.to(device))
gnn_ap = average_precision_score(test_data.edge_label.cpu().numpy(), torch.sigmoid(out_gnn).cpu().detach().numpy())

#Comparison
print("FINAL MODEL PERFORMANCE COMPARISON")
results_df = pd.DataFrame([
    {'Model': 'Node2Vec', 'Test AUC': n2v_auc, 'Test AP': n2v_ap},
    {'Model': 'GNN (GAT)', 'Test AUC': gnn_auc, 'Test AP': gnn_ap}
])
print(results_df.to_string(index=False, float_format='%.4f'))

Training and Evaluating Model 1: Node2Vec
Training Node2Vec model
Node2Vec training complete.
Training a classifier on Node2Vec embeddings
Classifier training complete.
Training and Evaluating Model 2: GNN (Graph Attention Network)
Epoch: 020, Loss: 0.5090, Val AUC: 0.8303
Epoch: 040, Loss: 0.4922, Val AUC: 0.8609
Epoch: 060, Loss: 0.4501, Val AUC: 0.8935
Epoch: 080, Loss: 0.3907, Val AUC: 0.9064
Epoch: 100, Loss: 0.3765, Val AUC: 0.9232
Epoch: 120, Loss: 0.3075, Val AUC: 0.9438
Epoch: 140, Loss: 0.3051, Val AUC: 0.9459
Epoch: 160, Loss: 0.2865, Val AUC: 0.9470
Epoch: 180, Loss: 0.2959, Val AUC: 0.9472
Epoch: 200, Loss: 0.2689, Val AUC: 0.9496
FINAL MODEL PERFORMANCE COMPARISON
    Model  Test AUC  Test AP
 Node2Vec    0.8337   0.8162
GNN (GAT)    0.9618   0.9628
